In [2]:
import cv2
from tqdm import tqdm
import os

In [4]:
def process_and_save_image(image_directory, save_directory, square_size):
    # Read the image
    image = cv2.imread(image_directory)
    
    if image is None:
        # print(f"No image found in {image_directory}")
        return
    
    # Get dimensions of the image
    height, width, _ = image.shape
    
    # Crop the image to a square size
    # Determine the center to crop the image around
    center_x, center_y = width // 2, height // 2
    x_start = max(center_x - square_size // 2, 0)
    y_start = max(center_y - square_size // 2, 0)
    
    # Ensuring the square doesn't exceed image dimensions
    cropped_image = image[y_start:y_start+square_size, x_start:x_start+square_size]
    
    # Flip the image vertically
    flipped_image = cv2.flip(cropped_image, 0)
    
    # Check if save directory exists, if not create it
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # Save the image
    filename = os.path.basename(image_directory)
    save_path = os.path.join(save_directory, filename)
    cv2.imwrite(save_path, flipped_image)
    # print(f"Processed image saved to {save_path}")

def process_directories(directories, square_size):
    for dir in directories:
        # Extract the prefix from the directory name
        parent_dir = os.path.abspath(os.path.join(dir, os.pardir))
        prefix = os.path.basename(parent_dir.rstrip('/')).split('_')[0]
        
        shrunk_dir_name = f"{prefix}_shrunk"
        shrunk_dir_path = os.path.join(parent_dir, shrunk_dir_name)

        # Process each file in the directory
        for file in tqdm([file for file in os.listdir(dir) if not file.startswith('.')], desc=prefix):
            process_and_save_image(os.path.join(dir, file), shrunk_dir_path, square_size)


# dir = '/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv1_frames/frames/'


# for file in tqdm([file for file in os.listdir(dir) if not file.startswith('.')]):
#     process_and_save_image(dir + file, "/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv1_frames/sv1_shrunk/", 480)

In [37]:
directories = [
    # '/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv1_frames/frames/',
    '/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv3_frames/frames/',
    '/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv4_frames/frames/',
    '/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv5_frames/frames/',
    '/home/nripstein/Documents/thesis data/thesis labels/sv labels/sv_extra_frames/frames/',
]

square_size = 480
process_directories(directories, square_size)

sv3


sv3: 100%|██████████| 5348/5348 [00:24<00:00, 215.57it/s]


sv4


sv4: 100%|██████████| 6974/6974 [00:32<00:00, 215.49it/s]


sv5


sv5: 100%|██████████| 3845/3845 [00:17<00:00, 213.92it/s]


sv


sv: 100%|██████████| 1875/1875 [00:08<00:00, 212.83it/s]


In [6]:
# directories = [
#     '/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr1/frames/',
#     '/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr3/frames/',
#     '/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr4/frames/',
#     '/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr_extra1/frames/',
#     '/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr_extra2/frames/',
# ]

directories = ["/home/nripstein/Documents/thesis data/thesis labels/nr labels/all_nr/frames"]

square_size = 480
process_directories(directories, square_size)

all: 100%|██████████| 14337/14337 [01:12<00:00, 197.64it/s]


# Make vid from predicted frames

In [21]:
import cv2
import os
import numpy as np
from numpy.typing import ArrayLike
from tqdm import tqdm
import pandas as pd

def sorting_fn(filename):
    return int(filename.split("_")[0])


def images_to_video(input_dir: str, output_file: str, fps = 60, labels: ArrayLike = None):

    red = [52, 28, 200]
 
    green = [47, 140, 38] # in
    label_color_map = {0: red, 1: green}

    all_images = [f for f in os.listdir(input_dir) if f[0] != "."]
    sorted_images = sorted(all_images, key=lambda filename: int(filename.split("_")[0]))

    if labels is not None and len(labels) != len(sorted_images):
        raise IndexError("Label array must be same length as number of frames")

    # Read the first image to get its resolution
    first_image_path = os.path.join(input_dir, sorted_images[0])
    first_image = cv2.imread(first_image_path)
    height, width, _ = first_image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_out = cv2.VideoWriter(output_file, fourcc, fps, (480, 480))  # Change resolution if necessary

    for idx, image_file in enumerate(tqdm(sorted_images)):
        # Extract frame number from the image filename
        frame_number = int(image_file.split('_')[0])

        # Read the image
        img = cv2.imread(os.path.join(input_dir, image_file))

        if labels is not None:
            label = labels[idx]  # label of current frame
            cv2.rectangle(img, (0, 0), (80, 80), label_color_map[label], -1)

        # Write the frame to the video
        video_out.write(img)

    # Release video writer and close any open windows
    video_out.release()
    cv2.destroyAllWindows()

# # Example usage:
base = '/home/nripstein/code/thesis git clones/hand_object_detector/'
input_dir = base + 'images_det/'
output_file = base + 'output_video1.mp4'


GOPR4366_pred = pd.read_csv('/home/nripstein/Documents/thesis data/lab participants/p6/GOPR4366_pred.csv' )
GOPR4366_pred_bin = GOPR4366_pred["contact_label_pred"].map({"Portable Object": 1, "No Contact": 0 , "Stationary Object Contact": 0, "Self Contact": 0}).to_numpy().astype(np.int8)

sv2_con1 = pd.read_csv("/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr2/sr2_pred_con1.csv")
sv2_con1_pred_bin = sv2_con1["contact_label_pred"].map({"Portable Object": 1, "No Contact": 0 , "Stationary Object Contact": 0, "Self Contact": 0}).to_numpy().astype(np.int8)


# images_to_video(input_dir, output_file, 60, sv2_con1_pred_bin)

sv2_con1_og = pd.read_csv("/home/nripstein/Documents/thesis data/thesis labels/sr labels/sr2/sr2_pred.csv")
sv2_con1_pred_bin = sv2_con1["contact_label_pred"].map({"Portable Object": 1, "No Contact": 0 , "Stationary Object Contact": 0, "Self Contact": 0}).to_numpy().astype(np.int8)

images_to_video(input_dir, output_file, 60, sv2_con1_pred_bin)


  0%|          | 0/4278 [00:00<?, ?it/s]

100%|██████████| 4278/4278 [00:13<00:00, 328.96it/s]
